In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Loading the dataset (using the downloaded file)
ocean = pd.read_csv("Data_OceanCleanup.csv")

In [3]:
# Displaying the initial 5 records, to check whether the data has "NaN" values
ocean.head()

,Cleanup ID,Zone,State,Country,GPS,Cleanup Type,Cleanup Date,Group Name,Adults,Children,...,Syringes,Tampons/Tampon Applicators,Personal Hygiene (Clean Swell),Foam Pieces,Glass Pieces,Plastic Pieces,Total Items Collected,Trash level,Immediate Cleanup Required?,Immediate Cleanup Required
0,86,"Cuyahoga County, OH, USA","Ohio, USA",United States,"41.49742, -81.56505","Land (beach, shoreline and inland)",12/22/14,NaN,1.0,0.0,...,0,0,0,0,0,0,3.0,Low,NaN,No
1,87,"Fairfax County, VA, USA","Virginia, USA",United States,"38.87456, -77.24428","Land (beach, shoreline and inland)",12/24/14,NaN,1.0,0.0,...,0,0,0,0,0,2,10.0,Low,NaN,No
2,92,"Washington, DC, USA","District of Columbia, USA",United States,"38.90733, -77.04405","Land (beach, shoreline and inland)",2/19/15,NaN,1.0,0.0,...,0,0,0,0,0,0,4.0,Low,NaN,No
3,93,"Washington, DC, USA","District of Columbia, USA",United States,"38.90733, -77.0441","Land (beach, shoreline and inland)",2/19/15,NaN,1.0,0.0,...,0,0,0,0,0,0,14.0,Low,NaN,No
4,94,"Washington, DC, USA","District of Columbia, USA",United States,"38.90733, -77.0441","Land (beach, shoreline and inland)",2/19/15,NaN,1.0,0.0,...,0,0,0,0,0,0,14.0,Low,NaN,No


In [4]:
ocean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37904 entries, 0 to 37903
Data columns (total 64 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Cleanup ID                                 37904 non-null  object 
 1   Zone                                       37665 non-null  object 
 2   State                                      37577 non-null  object 
 3   Country                                    37903 non-null  object 
 4   GPS                                        37903 non-null  object 
 5   Cleanup Type                               37903 non-null  object 
 6   Cleanup Date                               37903 non-null  object 
 7   Group Name                                 26636 non-null  object 
 8   Adults                                     37903 non-null  float64
 9   Children                                   37903 non-null  float64
 10  People                

### Data Cleaning

In [5]:
# Checking for blanks (empty strings) in all columns
for column in ocean.columns:
    blank_values = ocean[column].apply(lambda x: str(x).strip() == '').sum()
    print(f"Blanks in '{column}': {blank_values}")

Blanks in 'Cleanup ID': 0
Blanks in 'Zone': 0
Blanks in 'State': 0
Blanks in 'Country': 0
Blanks in 'GPS': 0
Blanks in 'Cleanup Type': 0
Blanks in 'Cleanup Date': 0
Blanks in 'Group Name': 26
Blanks in 'Adults': 0
Blanks in 'Children': 0
Blanks in 'People': 0
Blanks in 'Pounds': 0
Blanks in 'Miles': 0
Blanks in '# of bags': 0
Blanks in 'Cigarette Butts': 0
Blanks in 'Food Wrappers (candy, chips, etc.)': 0
Blanks in 'Take Out/Away Containers (Plastic)': 0
Blanks in 'Take Out/Away Containers (Foam)': 0
Blanks in 'Bottle Caps (Plastic)': 0
Blanks in 'Bottle Caps (Metal)': 0
Blanks in 'Lids (Plastic)': 0
Blanks in 'Straws, Stirrers': 0
Blanks in 'Forks, Knives, Spoons': 0
Blanks in 'Beverage Bottles (Plastic)': 0
Blanks in 'Beverage Bottles (Glass)': 0
Blanks in 'Beverage Cans': 0
Blanks in 'Grocery Bags (Plastic)': 0
Blanks in 'Other Plastic Bags': 0
Blanks in 'Paper Bags': 0
Blanks in 'Cups, Plates (Paper)': 0
Blanks in 'Cups, Plates (Plastic)': 0
Blanks in 'Cups, Plates (Foam)': 0
Blank

___

### 1. "Cleanup Type" column: Removing the 'blank' value in "Cleanup Type" (there is only 1 blank row showing the vertical count of values)

In [6]:
ocean = ocean.dropna(subset=['Cleanup Type'])

___

### 2. "Cleanup Date" column: Deleting the record which has date "0218-04-20" as it will cause issue during our analysis & further cleanup

In [7]:
# Defining the date to be deleted
date_to_delete = "0218-04-20"

# Using the boolean indexing to filter out the record with the specified date
ocean = ocean[ocean['Cleanup Date'] != date_to_delete]

___

### 3. "Group Name" column: Adding a prefix "Group_" to the existing Group Names

In [8]:
# Defining a function to add the "Group_" prefix and handle blank values
def add_grp_prefix(row):
    if pd.isna(row['Group Name']) or row['Group Name'] == ' ' or row['Group Name'] == '    ':
        return f'Group_{row["Cleanup Date"]}'
    else:
        return f'Group_{row["Group Name"]}'

In [9]:
# Applying the function to the "Group Name" column
ocean['Group Name'] = ocean.apply(add_grp_prefix, axis=1)

### 4. "Zone" column: Removing the additional values (state name and country name after zone name)

In [10]:
# Splitting the "Zone" column by ','
ocean['Zone'] = ocean['Zone'].str.split(',').str[0]

___


### 5. "Country" column: Removing the additional value (country name after state name)

In [11]:
# Splitting the "State" column by ','
ocean['State'] = ocean['State'].str.split(',').str[0]

### 6. Replacing null valuesin  Zone  based on State 

In [12]:
def standardize_zone_name(zone):
    if pd.isnull(zone):
        return zone  # Return NaN as it is
    zone = zone.strip()  # Remove any leading/trailing whitespace
    if "County" not in zone:
        zone += " County"  # Append "County" if it's not already there
    return zone

# Apply the function to the 'Zone' column
ocean['Zone'] = ocean['Zone'].apply(standardize_zone_name)




In [13]:
# Create a mapping from Zone to State based on the most common State for each Zone
state_mapping = ocean.groupby('Zone')['State'].agg(pd.Series.mode).to_dict()

# Function to apply the mapping
def fill_missing_state(row):
    if pd.isna(row['State']) and row['Zone'] in state_mapping:
        return state_mapping[row['Zone']]
    else:
        return row['State']

# Apply the function
ocean['State'] = ocean.apply(fill_missing_state, axis=1)




### 7. Stadardizing the State and zone data

In [14]:
# Create a mapping from GPS to State and Zone
gps_state_mapping = ocean.dropna(subset=['State']).set_index('GPS')['State'].to_dict()
gps_zone_mapping = ocean.dropna(subset=['Zone']).set_index('GPS')['Zone'].to_dict()

# Function to apply the mappings to fill missing values
def fill_missing_by_gps(row, gps_mapping):
    if pd.isna(row['State']) and row['GPS'] in gps_mapping:
        return gps_mapping[row['GPS']]
    else:
        return row['State']

def fill_missing_zone_by_gps(row, gps_mapping):
    if pd.isna(row['Zone']) and row['GPS'] in gps_mapping:
        return gps_mapping[row['GPS']]
    else:
        return row['Zone']

# Apply the function
ocean['State'] = ocean.apply(fill_missing_by_gps, axis=1, gps_mapping=gps_state_mapping)
ocean['Zone'] = ocean.apply(fill_missing_zone_by_gps, axis=1, gps_mapping=gps_zone_mapping)




### 8. Replaced state and zone based on GPS location

In [15]:
# Assuming GPS coordinates are in the format 'latitude longitude' as strings
# We will split them and round each to a precision that still ensures uniqueness in your dataset
# Here we use 2 decimal places as an example. You may adjust the precision as needed.

# Function to round the GPS coordinates
def round_gps(gps, decimals=2):
    if pd.isna(gps) or isinstance(gps, float):
        return None  # Return None for NaN or non-string values
    try:
        lat, lon = map(str.strip, gps.split(' '))
        lat, lon = round(float(lat), decimals), round(float(lon), decimals)
        return f"{lat} {lon}"
    except ValueError:  # In case there's any error in conversion
        return None

# Apply the rounding function to the GPS column
ocean['Rounded_GPS'] = ocean['GPS'].apply(round_gps)

# Create mappings of rounded GPS to 'State' and 'Zone'
gps_state_mapping = ocean.dropna(subset=['State']).set_index('Rounded_GPS')['State'].to_dict()
gps_zone_mapping = ocean.dropna(subset=['Zone']).set_index('Rounded_GPS')['Zone'].to_dict()

# Function to apply the mappings to fill missing values
def fill_missing_by_rounded_gps(row, gps_mapping):
    if pd.isna(row['State']) and row['Rounded_GPS'] in gps_mapping:
        return gps_mapping[row['Rounded_GPS']]
    else:
        return row['State']

def fill_missing_zone_by_rounded_gps(row, gps_mapping):
    if pd.isna(row['Zone']) and row['Rounded_GPS'] in gps_mapping:
        return gps_mapping[row['Rounded_GPS']]
    else:
        return row['Zone']

# Apply the function to fill missing 'State' and 'Zone' values
ocean['State'] = ocean.apply(fill_missing_by_rounded_gps, axis=1, gps_mapping=gps_state_mapping)
ocean['Zone'] = ocean.apply(fill_missing_zone_by_rounded_gps, axis=1, gps_mapping=gps_zone_mapping)

# Drop the Rounded_GPS column if not needed
ocean.drop(columns=['Rounded_GPS'], inplace=True)

### 9. Dividing Longitude and Latitude columns separately from GPS 

In [16]:
# Assuming 'ocean' is your DataFrame after label encoding other categorical columns

# Split the GPS column into two new columns 'Latitude' and 'Longitude'
ocean[['Latitude', 'Longitude']] = ocean['GPS'].str.split(',', expand=True)

# Convert the new columns to numeric data type
ocean['Latitude'] = pd.to_numeric(ocean['Latitude'])
ocean['Longitude'] = pd.to_numeric(ocean['Longitude'])

# Optionally, round the latitude and longitude to a desired level of precision
ocean['Latitude'] = ocean['Latitude'].round(6)  # Adjust the number of decimal places as needed
ocean['Longitude'] = ocean['Longitude'].round(6)  # Adjust the number of decimal places as needed

# Now you can drop the original 'GPS' column if it's no longer needed
ocean.drop(columns=['GPS'], inplace=True)

# Continue with any further processing and then save or use the DataFrame for modeling


### 10. Dividing Date ,month and year  columns separately from Date  

In [17]:
# Convert 'Cleanup Date' to datetime
ocean['Cleanup Date'] = pd.to_datetime(ocean['Cleanup Date'])

# Extract year, month, and day as separate features
ocean['Year'] = ocean['Cleanup Date'].dt.year
ocean['Month'] = ocean['Cleanup Date'].dt.month
ocean['Day'] = ocean['Cleanup Date'].dt.day

# Now you can drop the original 'Cleanup Date' column if it's no longer needed
ocean.drop(columns=['Cleanup Date'], inplace=True)


### 11. Converting "Trash Level" to binary column

In [18]:
# Print unique values before replacement
print("Before Replacement:")
print(ocean['Trash level'].unique())

# Map 'High' to 1 and 'Medium'/'Low' to 0 in the existing column
ocean['Trash level'].replace({'High': 2, 'Medium': 1, 'Low': 0}, inplace=True)

# Print unique values after replacement
print("\nAfter Replacement:")
print(ocean['Trash level'].unique())


Before Replacement:
['Low' 'Medium' 'High']

After Replacement:
[0 1 2]


____

___

### 12. Converting "Immediate Cleanup Required" to binary column 

In [19]:
# Print unique values before replacement
print("Before Replacement:")
print(ocean['Immediate Cleanup Required'].unique())

# Map 'Yes' to 1 and 'No' to 0 in the existing column
ocean['Immediate Cleanup Required'].replace({'Yes': 1, 'No': 0}, inplace=True)

# Print unique values after replacement
print("\nAfter Replacement:")
print(ocean['Immediate Cleanup Required'].unique())

Before Replacement:
['No' 'Yes']

After Replacement:
[0 1]


___

### 13. Dropping the non-numeric rows in Cleanup ID

In [20]:
# Check if 'Cleanup ID' contains non-numeric values
non_numeric_rows = ocean[pd.to_numeric(ocean['Cleanup ID'], errors='coerce').isna()]

# Drop rows with non-numeric 'Cleanup ID'
ocean = ocean[~pd.to_numeric(ocean['Cleanup ID'], errors='coerce').isna()]

ocean.drop(columns=['Immediate Cleanup Required?'], inplace=True)

In [21]:
# Save the updated DataFrame to a new CSV file
ocean.to_csv('cleaned_ocean.csv', index=False)  # Replace 'updated_dataset.csv' with your desired output file name

___